## Import libraries

In [1]:
# import torch utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import os

# import transformers utils
from transformers import GPT2Tokenizer, GPT2Config, GPT2ForSequenceClassification, GPT2TokenizerFast

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# nvidia-smi that clears gpu memory
torch.cuda.empty_cache()

## Prepare data

In [2]:
from operations.utils import read_text

def read_text(path:str="./transcripts/corpus", file_name: str="corpus.txt"):
    with open(f'{path}/{file_name}', 'r', encoding='utf-8') as f:
        texts = f.read()
    return texts

In [3]:
# Tokenize
from operations.data import tokenize_text
from config.config import logger

def tokenize_text(text, model_name: str="gpt2", padding_side: str="right", context_length: int=256, tokenizer_only=False):
    
    # Instantiate tokenizer and pass `gpt2` to the `from_pretrained` method 
    tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
    # Select token to uses as `pad_token`
    tokenizer.pad_token = tokenizer.eos_token
    # Default to right padding
    tokenizer.padding_side = padding_side
    # Set context length
    context_length = context_length
    
    if tokenizer_only == True:
        return tokenizer
    
    # Process text
    inputs = tokenizer(text, 
                       padding='longest',
                       truncation=True,
                       return_tensors="pt",
                       max_length=context_length, # context size GPT-2: 1,024, GPT-3: 2,048
                       return_overflowing_tokens=True, # tokenize input and split into chunks
                       return_length=True, # return length of each created chunk
                       return_special_tokens_mask=True
                      )
    
    logger.info(f"Input IDs length: {len(inputs['input_ids'])}")
    logger.info(f"Input chunk lengths: {(inputs['length'])}")
    logger.info(f"Chunk mapping: {inputs['overflow_to_sample_mapping']}")

    return inputs
    
inputs = tokenize_text(read_text())

[09/30/22 10:22:58] INFO     Input IDs length: 32583                         ]8;id=377791;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py\1229640824.py]8;;\:]8;id=998416;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py#30\30]8;;\

                    INFO     Input chunk lengths: tensor([256, 256, 256,     ]8;id=311103;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py\1229640824.py]8;;\:]8;id=893235;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py#31\31]8;;\
                             ..., 256, 256, 256])                                            

                    INFO     Chunk mapping: tensor([0, 0, 0,  ..., 0, 0, 0]) ]8;id=804317;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py\1229640824.py]8;;\:]8;id=44041;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\1229640824.py#32\32]8;;\

In [4]:
# Train/Dev split
from typing import List, Dict
from operations.data import data_split

def data_split(tokens: Dict , train_split: float=0.8, validation_split: float=0.9):
    assert "input_ids" and "attention_mask" in tokens.keys()
    
    n1 = int(train_split * len(tokens['input_ids']))
    n2 = int(validation_split * len(tokens['input_ids']))
    
    Xtr = tokens['input_ids'][:n1]
    tr_mask = tokens['attention_mask'][:n1]
    
    Xval = tokens['input_ids'][n1:n2]
    val_mask = tokens['attention_mask'][n1:n2]
    
    logger.info(f"{Xtr.shape=}")
    logger.info(f"{Xval.shape=}")
    
    return Xtr, tr_mask, Xval, val_mask


Xtr, tr_mask, Xval, val_mask = data_split(inputs)

[09/30/22 10:23:01] INFO     Xtr.shape=torch.Size([26066, 256])              ]8;id=899041;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2131086292.py\2131086292.py]8;;\:]8;id=817867;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2131086292.py#17\17]8;;\

                    INFO     Xval.shape=torch.Size([3258, 256])              ]8;id=793487;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2131086292.py\2131086292.py]8;;\:]8;id=881090;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2131086292.py#18\18]8;;\

In [5]:
# Date Staging
# from operations.data import DataLoads

class DataLoads(Dataset):
    
    def __init__(self, X, Mask):
        self.x = X
        self.mask = Mask
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {
            'input_ids':self.x[idx],
            'attention_mask':self.mask[idx],
        }
    
def stage_data(xtrain: torch.tensor, 
               xtrain_mask: torch.tensor, 
               xval: torch.tensor, 
               xval_mask: torch.tensor):
    
    train_loader = DataLoads(xtrain, xtrain_mask)
    val_loader = DataLoads(xval, xval_mask)
    
    return train_loader, val_loader

In [6]:
# Prepare data for input to transformer
tr_loader, dev_loader = stage_data(Xtr, tr_mask, Xval, val_mask)

trainset = DataLoader(tr_loader, shuffle=True, batch_size=3)
devset = DataLoader(dev_loader, shuffle=False, batch_size=3)

In [10]:
# Model
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

def get_tokenizer(model_name: str="gpt2", padding_side:str="right"):
    # Instantiate tokenizer and pass `gpt2` to the `from_pretrained` method 
    tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
    # Select token to uses as `pad_token`
    tokenizer.pad_token = tokenizer.eos_token
    # Default to right padding
    tokenizer.padding_side = padding_side
    
    logger.info(tokenizer)
    
    return tokenizer

tokenizer = get_tokenizer()

context_length = 256

config = AutoConfig.from_pretrained(
    "gpt2", vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

[09/30/22 10:25:54] INFO     PreTrainedTokenizerFast(name_or_path='gpt2',    ]8;id=409574;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2645677773.py\2645677773.py]8;;\:]8;id=398606;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\2645677773.py#12\12]8;;\
                             vocab_size=50257, model_max_len=1024,                           
                             is_fast=True, padding_side='right',                             
                             truncation_side='right',                                        
                             special_tokens={'bos_token': '<|endoftext|>',                   
                             'eos_token': '<|endoftext|>', 'unk_token':                      
                             '<|endoftext|>', 'pad_token': '<|endoftext|>'})                 

In [8]:
model = GPT2LMHeadModel.from_pretrained("gpt2",config=config)
model_size = sum(t.numel() for t in model.parameters())
logger.info(f"The model has {model_size/1000**2:.1f}M parameters to tune")

[09/30/22 10:23:34] INFO     The model has 124.4M parameters to tune          ]8;id=512693;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\4012617785.py\4012617785.py]8;;\:]8;id=41555;file://C:\Users\Hedronstone\AppData\Local\Temp\ipykernel_4676\4012617785.py#3\3]8;;\

In [ ]:
# create data collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
# Optimization Loop
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="artifacts",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=10,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=200,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
model=model,
tokenizer=tokenizer,
args=args,
data_collator=data_collator,
train_dataset=tr_loader,
eval_dataset=dev_loader)

In [ ]:
trainer.train()

In [ ]:
# Inference
# How well does my trained model actually perform?
import torch
from transformers import pipeline

pipe = pipeline(
"text-generation", model="artifacts/checkpoint-200")

txt = "Artificial Intelligence will"
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

In [ ]:
txt = "Artificial Intelligence will"
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

In [ ]:
# ========== Advanced Training Loop ================
# Since I'm primarily interested in autocompletion for prompts around meaning,
# I will give more weight to training samples with "meaning" and its synonyms.
# Let's check for these words' existence in the tokenizer vocabulary

keytoken_ids = []
for keyword in [
    "connotation",
    "content",
    "context",
    "definition",
    "effect",
    "essence",
    "explanation",
    "hint",
    "implication",
    "interpretation",
    "nuance",
    "sense",
    "significance",
    "spirit",
    "purpose",
    "direction",
    "subject",
    "substance",
    "understanding",
    "value",
    "intention",
    "aim",
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
        keytoken_ids.append(ids[0])
    else:
        print(f"Keyword has not single token: {keyword}")

In [ ]:
### You need to study the below code. You initially copied and pasted it into the notebook cell. 
### It's value is such that closer inspection is warranted, and will pay dividends later in the form
### of knowledge which can be applied to future problems.

from torch.nn import CrossEntropyLoss
import torch


def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
        axis=[0, 2]
    )
    weights = alpha * (1.0 + weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss

In [ ]:
from torch.utils.data.dataloader import DataLoader

train_dataloader = DataLoader(tr_loader, batch_size=32, shuffle=True)
eval_dataloader = DataLoader(dev_loader, batch_size=32)

In [ ]:
weight_decay = 0.1

def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [ ]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [ ]:
model = GPT2LMHeadModel(config)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(model), lr=5e-4)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(fp16=True)

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

In [ ]:
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
eval_steps = 5000

model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=num_training_steps
    ):
        logits = model(batch["input_ids"]).logits
        loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        if step % 100 == 0:
            accelerator.print(
                {
#                     "lr": get_lr(),
#                     "samples": step * samples_per_step,
#                     "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(output_dir)
                repo.push_to_hub(
                    commit_message=f"Training in progress step {step}", blocking=False
                )

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
"text-generation", model="artifacts/checkpoint-200")

txt = "Artificial intelligence"
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])